# Extracción de Notas del Simulacro de la UNMSM

## Extracción de URLs

### Importar Librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

### Iniciar Microsoft Edge

In [2]:
driver = webdriver.Edge()

### Ingreso a la página del simulacro de la UNMSM

In [3]:
web_site = "https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1.html"
driver.get(web_site)

In [4]:
escuelas = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')

### Encontrar todos los elementos
* `driver`: Es el objeto de Selenium que controla el navegador web. A través de él, realizas todas las operaciones de automatización en la página web.

* `find_elements`: Es un método de Selenium que busca y devuelve una lista de todos los elementos que coinciden con el selector proporcionado. A diferencia de find_element, que devuelve solo el primer elemento encontrado, find_elements devuelve una lista con todos los elementos que coinciden.

* `By.CSS_SELECTOR`: Es un método para localizar elementos en el DOM (Document Object Model) usando selectores CSS. Los selectores CSS son una forma muy flexible y potente de seleccionar elementos en una página web.

* 'tbody tr': Este es el selector CSS utilizado para encontrar los elementos.
    * tbody selecciona el cuerpo de una tabla HTML, que es el contenedor de las filas.
    * tr selecciona las filas dentro del tbody.

![Página Principal](Imagenes/primero.png)

In [5]:
# Creacion de lista para guardar los URLs
Escuelas_Profesionales = []

### Iteración por escuela.
* Encontrar un enlace (`<a>`) dentro del elemento.
* Utilizamos `By.TAG_NAME` para encontrar el `<a>`.
* Extraer la URL del enlace (`href`) y el texto visible del enlace.
* Guardamos la información en un diccionario en una lista, para tenerlo en un formato JSON.
  
![Página Principal](Imagenes/segundo.png)

In [6]:
for escuela in escuelas:
    try:
        url = escuela.find_element(By.TAG_NAME, 'a').get_attribute('href')
        ep = escuela.find_element(By.TAG_NAME, 'a').text
        codigo = url.replace("https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/", "")
        codigo = codigo.replace("/0.html", "")
        Escuelas_Profesionales.append({
            'Codigo Ep': codigo,
            'Escuela': ep,
            'URL': url
        })
    except NoSuchElementException:
        print("Enlace no encontrado en una de las filas.")

In [7]:
Escuelas_Profesionales

[{'Codigo Ep': '091',
  'Escuela': 'ADMINISTRACIÓN',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/091/0.html'},
 {'Codigo Ep': '094',
  'Escuela': 'ADMINISTRACIÓN DE LA GASTRONOMÍA',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/094/0.html'},
 {'Codigo Ep': '093',
  'Escuela': 'ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/093/0.html'},
 {'Codigo Ep': '092',
  'Escuela': 'ADMINISTRACIÓN DE TURISMO',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/092/0.html'},
 {'Codigo Ep': '095',
  'Escuela': 'ADMINISTRACIÓN MARÍTIMA Y PORTUARIA',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/095/0.html'},
 {'Codigo Ep': '153',
  'Escuela': 'ANTROPOLOGÍA',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/153/0.html'},
 {'Codigo Ep': '154',
  'Escuela': 'ARQUEOLOGÍA',
  'URL': 'https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/154/0.html'},
 {'Codigo 

In [8]:
driver.quit()

## Extracción de Notas por Escuela

In [103]:
data_estudiantes = []

In [94]:
from selenium.webdriver.common.action_chains import ActionChains

In [98]:
def extraccion_data(link):
    """
    Extrae datos de estudiantes de una tabla paginada en un sitio web.

    Esta función navega a un enlace proporcionado, extrae información sobre los estudiantes
    de una tabla paginada y la almacena en una lista global llamada `notas`. Recorre todas
    las páginas de la tabla hasta que no hay más páginas disponibles.

    Parámetros:
    ----------
    link : str
        URL de la página web que contiene la tabla de estudiantes.

    Comportamiento:
    --------------
    - Navega al enlace proporcionado y espera 3 segundos para asegurar que la página haya cargado.
    - Encuentra todos los estudiantes en la página actual.
    - Para cada estudiante, extrae la Escuela Profesional (columna 4) y la Nota (columna 5).
    - Almacena la información extraída en una lista global `notas`.
    - Continúa navegando a la siguiente página de la tabla hasta que ya no hay más páginas.
    
    Excepciones manejadas:
    ---------------------
    - NoSuchElementException: Ignora los estudiantes que no tienen la estructura esperada.

    Nota:
    -----
    - La función utiliza la variable global `notas` para almacenar los resultados. Asegúrate de 
      que esta variable esté definida antes de llamar a la función.
    """
    driver.get(link)
    time.sleep(3)
    ##Acciones para Ejecutar la pagina correctamente
    
    # Realiza un scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Espera un momento para que el contenido cargue

    # Simula un clic en un punto de la página para activar los elementos
    action = ActionChains(driver)
    action.move_by_offset(10, 10).click().perform()
    
    time.sleep(1)
    while True:
        # Encuentra todos los estudiantes en la página actual
        estudiantes = driver.find_elements(By.CSS_SELECTOR, '#tablaPostulantes tbody tr')
        # Recorre los estudiantes y extrae la información
        for i, estudiante in enumerate(estudiantes):
            try:
                ep = estudiante.find_element(By.CSS_SELECTOR, f'td:nth-child(4)')
                nota = estudiante.find_element(By.CSS_SELECTOR, f'td:nth-child(5)')
                data_estudiantes.append({
                    'Escuela Profesional': ep.text,
                    'Nota': nota.text
                })
            except NoSuchElementException:
                pass
        # Encuentra el botón de la siguiente página
        next_button = driver.find_element(By.CSS_SELECTOR, '#tablaPostulantes_next > a')
        alerta = driver.find_element(By.CSS_SELECTOR, '#tablaPostulantes_next')
        
        if 'disabled' in alerta.get_attribute('class'):
            break
        else:
            next_button.click()

## Búsqueda de Escuela Profesional

In [102]:
def buscar_escuela(codigo):
    """
    Busca la información de una escuela profesional basada en el código ingresado.

    Esta función recorre una lista de diccionarios llamada `Escuelas_Profesionales` para buscar
    el diccionario que contiene un código de escuela específico. Si el código se encuentra, 
    se obtiene la URL y el nombre de la escuela, se reemplazan los espacios en blanco en el nombre
    por guiones bajos, y se procede a extraer la información de la URL. Finalmente, cierra el driver.

    Args:
        codigo (str): El código de la escuela profesional que se desea buscar.

    Variables locales:
        url_escuela (str or None): La URL de la escuela profesional si el código es encontrado.
        nombre_escuela (str or None): El nombre de la escuela profesional con los espacios
                                      reemplazados por guiones bajos si el código es encontrado.

    Funciones llamadas:
        extraccion_data(url_escuela): Función que extrae la información de la URL encontrada.
        driver.quit(): Cierra el navegador después de la extracción de datos.

    Prints:
        Si el código es encontrado, imprime la URL de la escuela profesional.
        Si el código no es encontrado, imprime un mensaje de error.

    Example:
        >>> buscar_escuela("001")
        La URL de la escuela profesional es: https://ejemplo.com/sistemas
        (Se ejecuta la extracción de datos y se cierra el driver)
    """
    url_escuela = None
    nombre_escuela = None

    for escuela_profesional in Escuelas_Profesionales:
        if escuela_profesional["Codigo Ep"] == codigo:
            url_escuela = escuela_profesional['URL']
            nombre_escuela = escuela_profesional['Escuela'].replace(" ", "_")
            break

    if url_escuela:
        print(f"La URL de la escuela profesional es: {url_escuela}")
        print(f"El nombre de la escuela profesional es: {nombre_escuela}")
        extraccion_data(url_escuela)
        return nombre_escuela
    else:
        print("El código ingresado no existe.")
        return None

In [104]:
driver = webdriver.Edge()

In [105]:
codigo = input("Ingrese el código de Escuela: ")
nombre_escuela = buscar_escuela(codigo)

Ingrese el código de Escuela:  091


La URL de la escuela profesional es: https://admision.unmsm.edu.pe/WebsiteSimulacro20251/1/091/0.html
El nombre de la escuela profesional es: ADMINISTRACIÓN


In [106]:
driver.quit()

## Guardar data y exportarlo en un archivo CSV

In [107]:
data_estudiantes = pd.DataFrame(data_estudiantes)
data_estudiantes.to_csv(f'Escuela_{nombre_escuela}.csv', index=False)